In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
import os
import matplotlib.pyplot as plt
import glob

%matplotlib inline

In [2]:
batch_size = 1

image_gen_train = ImageDataGenerator(rescale=1./255, 
                                     rotation_range=15, 
                                     width_shift_range=.15, 
                                     height_shift_range=.15, 
                                     horizontal_flip=True, 
                                     vertical_flip=True,
                                     zoom_range=0.2
                                    )


train_data_gen = image_gen_train.flow_from_directory(directory='./products_images/train',
                                                     class_mode='categorical', 
                                                     target_size=(256,256),
                                                     batch_size=batch_size)

Found 310 images belonging to 5 classes.


In [3]:
train_data_gen.class_indices

{'iphone12_pro_max': 0,
 'jbl_charge3': 1,
 'nintendo_switch': 2,
 'ps4_controller': 3,
 'yeezy_boost_350': 4}

In [4]:
image_gen_val = ImageDataGenerator(rescale=1./255)

val_data_gen = image_gen_val.flow_from_directory(directory='./products_images/validate', 
                                                   class_mode='categorical', 
                                                   target_size=(256,256),
                                                   batch_size=batch_size)

Found 113 images belonging to 5 classes.


In [5]:
val_data_gen.class_indices

{'iphone12_pro_max': 0,
 'jbl_charge3': 1,
 'nintendo_switch': 2,
 'ps4_controller': 3,
 'yeezy_boost_350': 4}

In [6]:
transfer_learning_layer = tf.keras.applications.MobileNetV2(include_top=False, 
                                                      weights='imagenet', 
                                                      input_shape=(256,256,3)
                                                     )

In [7]:
transfer_learning_layer.trainable = False

In [8]:
model = Sequential([
    transfer_learning_layer,
    Dropout(0.2),
    Flatten(),
    Dense(5, activation='softmax')
])

In [9]:
model.compile(optimizer='Adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy']
             )

In [10]:
epochs = 4
history = model.fit(train_data_gen, 
                    epochs=epochs, 
                    validation_data=val_data_gen)

Epoch 1/4
310/310 [==============================] - 23s 73ms/step - loss: 6.7870 - accuracy: 0.6871 - val_loss: 3.7665 - val_accuracy: 0.8584
Epoch 2/4
310/310 [==============================] - 22s 70ms/step - loss: 3.8050 - accuracy: 0.8806 - val_loss: 1.7038 - val_accuracy: 0.9558
Epoch 3/4
310/310 [==============================] - 22s 69ms/step - loss: 4.6176 - accuracy: 0.8581 - val_loss: 0.6452 - val_accuracy: 0.9381
Epoch 4/4
310/310 [==============================] - 22s 70ms/step - loss: 2.7806 - accuracy: 0.9258 - val_loss: 0.8788 - val_accuracy: 0.9646


In [11]:
model.save('./extracted_product_detection_model')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ../extracted_product_detection_model\assets


In [12]:
image_gen_test = ImageDataGenerator(rescale=1./255)

ps4_controller_test_data_gen = image_gen_test.flow_from_directory(directory='./products_images/test', 
                                                   class_mode='categorical', 
                                                   target_size=(256,256),
                                                   batch_size=batch_size)

Found 6 images belonging to 5 classes.


In [13]:
test_pred = model.predict(ps4_controller_test_data_gen)

In [14]:
test_pred_bool = (test_pred > 0.5)
test_predictions = test_pred_bool.astype(int)
test_predictions

array([[0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0]])